In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature = 0 , groq_api_key ="123", model_name = "llama-3.1-70b-versatile")
response = llm.invoke("the first person to land moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


### Using fireduck we can run our panda program faster.

### Using langchain , web base loader we can get the information from Website URL

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-41673")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, 

In [6]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Software Engineer, ITC",
  "experience": "6-9 years",
  "skills": [
    "Java",
    "Spring Boot",
    "Micro service architecture",
    "Domain driven design",
    "RESTful Services",
    "Test Driven development",
    "Junit",
    "Mockito",
    "Splunk",
    "SignalFx",
    "New Relic",
    "NoSQL data stores",
    "Cassandra",
    "DynamoDB",
    "MongoDB",
    "Relational data stores",
    "MySQL",
    "Oracle",
    "Continuous integration",
    "Unit testing",
    "Static analysis",
    "Automated integration tests",
    "Scrum",
    "Agile principles"
  ],
  "description": "As a Software Engineer for the Supply Chain and Planning Technology team, you will be part of a mission critical team focused on building a high-throughput, low-latency, fault-tolerant supply chain platform that drive Nike’s Digital transformation and growth strategies."
}
```


In [8]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer, ITC',
 'experience': '6-9 years',
 'skills': ['Java',
  'Spring Boot',
  'Micro service architecture',
  'Domain driven design',
  'RESTful Services',
  'Test Driven development',
  'Junit',
  'Mockito',
  'Splunk',
  'SignalFx',
  'New Relic',
  'NoSQL data stores',
  'Cassandra',
  'DynamoDB',
  'MongoDB',
  'Relational data stores',
  'MySQL',
  'Oracle',
  'Continuous integration',
  'Unit testing',
  'Static analysis',
  'Automated integration tests',
  'Scrum',
  'Agile principles'],
 'description': 'As a Software Engineer for the Supply Chain and Planning Technology team, you will be part of a mission critical team focused on building a high-throughput, low-latency, fault-tolerant supply chain platform that drive Nike’s Digital transformation and growth strategies.'}

In [9]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df.head(2)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio


In [10]:
import uuid
import chromadb
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name ="portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'], metadatas={'link':row['Links']}, ids=[str(uuid.uuid4())])

In [13]:
links = collection.query(query_texts=job['skills'],n_results=2)
links

{'ids': [['e749c089-843f-4354-a179-d8c739843da1',
   'd313ff8d-1112-42aa-95f4-84fd1f389d75'],
  ['e749c089-843f-4354-a179-d8c739843da1',
   'adc4430d-1cc6-46b0-ba96-9797e9169a9d'],
  ['d313ff8d-1112-42aa-95f4-84fd1f389d75',
   'c12f4511-7899-4a7e-a494-72f400ec55eb'],
  ['adc4430d-1cc6-46b0-ba96-9797e9169a9d',
   'd313ff8d-1112-42aa-95f4-84fd1f389d75'],
  ['e9275550-03c5-4007-aed9-c1df282642cb',
   'adc4430d-1cc6-46b0-ba96-9797e9169a9d'],
  ['e9cc6945-9eed-45e5-b450-b7d5ebb2d639',
   'd454da7e-8dce-43ec-97d9-8b6d18947ee3'],
  ['e749c089-843f-4354-a179-d8c739843da1',
   'adc4430d-1cc6-46b0-ba96-9797e9169a9d'],
  ['e749c089-843f-4354-a179-d8c739843da1',
   'd313ff8d-1112-42aa-95f4-84fd1f389d75'],
  ['5ca4ff08-f86d-4d53-bd57-2a1507cc9018',
   'd454da7e-8dce-43ec-97d9-8b6d18947ee3'],
  ['815fd477-7686-4cb5-9859-174ff72a46b6',
   'd454da7e-8dce-43ec-97d9-8b6d18947ee3'],
  ['e9cc6945-9eed-45e5-b450-b7d5ebb2d639',
   'adc4430d-1cc6-46b0-ba96-9797e9169a9d'],
  ['749f11f2-306e-432b-b316-29c03a3d

In [12]:
job= json_res
job['skills']

['Java',
 'Spring Boot',
 'Micro service architecture',
 'Domain driven design',
 'RESTful Services',
 'Test Driven development',
 'Junit',
 'Mockito',
 'Splunk',
 'SignalFx',
 'New Relic',
 'NoSQL data stores',
 'Cassandra',
 'DynamoDB',
 'MongoDB',
 'Relational data stores',
 'MySQL',
 'Oracle',
 'Continuous integration',
 'Unit testing',
 'Static analysis',
 'Automated integration tests',
 'Scrum',
 'Agile principles']

In [15]:
### create Prompt template form email
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rahul Gangahar, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)



Subject: Expertise in Building High-Throughput, Low-Latency Supply Chain Platforms

Dear Hiring Manager,

I came across the job description for a Senior Software Engineer, ITC at Nike, and I was impressed by the company's commitment to building a high-throughput, low-latency, fault-tolerant supply chain platform. As a Business Development Executive at AtliQ, I'd like to introduce our company's expertise in facilitating the seamless integration of business processes through automated tools.

Our team at AtliQ has extensive experience in developing scalable, process-optimized, and cost-reduced solutions for various industries. We've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our portfolio showcases our expertise in Java, Spring Boot, Microservices Architecture, Domain-Driven Design, RESTful Services, Test-Driven Development, and Continuous Integration, which aligns with the requi